# Q3


In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Construct credentials from service account key file
credentials = service_account.Credentials.from_service_account_file(
    'tche368-isom676-srvacct_srvacct.json')  # Update the file path as needed

# Construct a BigQuery client object
client = bigquery.Client(credentials=credentials)

# Revised and optimized query
QUERY = """
WITH product_filter AS (
    SELECT *
    FROM `machine_learning.products`
    WHERE prod_category NOT IN ("Gift Cards", "Other", "Front End Service", "Scanning Errors", "Customer Service-Misc", "Empties and Additionals")
),
valid_transactions AS (
    SELECT *
    FROM `machine_learning.transactions` a 
    join product_filter b on a.prod_id  = b.prod_id 
    WHERE trans_dt < "2020-03-01"
    AND a.prod_id IN (SELECT prod_id FROM product_filter)
        AND 
        -- Logic 1: Either sales_qty or sales_wgt is zero, but not both
        ((sales_qty = 0 AND sales_wgt <> 0) OR (sales_qty <> 0 AND sales_wgt = 0))
        AND 
        -- Logics 2 and 3 are parallel conditions
        (
            (prod_category NOT IN ("Coupon", "returns") AND (sales_qty > 0 OR sales_wgt > 0))
            OR
            (prod_category IN ("Coupon", "returns") AND (sales_qty < 0 OR sales_wgt < 0))
        )
    AND sales_amt >= 0
),
transactions_per_day AS (
    SELECT cust_id, trans_dt, COUNT(DISTINCT trans_id) AS trans_per_day
    FROM valid_transactions
    GROUP BY cust_id, trans_dt
    HAVING trans_per_day <= 10
),
eligible_custs AS (
    SELECT v.cust_id
    FROM valid_transactions v
    JOIN transactions_per_day tpd ON v.cust_id = tpd.cust_id AND v.trans_dt = tpd.trans_dt
    GROUP BY v.cust_id
    HAVING COUNT(DISTINCT v.trans_id) >= 5
    AND COUNT(DISTINCT v.trans_dt) >= 5
    AND COUNT(v.trans_id) <= 20000
),
sampled_custs AS (
    SELECT cust_id
    FROM eligible_custs
    WHERE MOD(ABS(FARM_FINGERPRINT(CAST(cust_id AS STRING))), 1000) < 1
)
SELECT tx.*
FROM `valid_transactions` tx
JOIN sampled_custs ON tx.cust_id = sampled_custs.cust_id
WHERE tx.trans_dt < "2020-03-01"
"""

# Execute the query
query_job = client.query(QUERY)  # API request

# Convert to DataFrame
sample_transaction = query_job.to_dataframe()  # Waits for query to finish and converts it to DataFrame


In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Construct credentials from service account key file
credentials = service_account.Credentials.from_service_account_file(
    'tche368-isom676-srvacct_srvacct.json')  # Update the file path as needed

# Construct a BigQuery client object
client = bigquery.Client(credentials=credentials)

# Revised and optimized query
QUERY = """
WITH product_filter AS (
    SELECT *
    FROM `machine_learning.products`
    WHERE prod_category NOT IN ("Gift Cards", "Other", "Front End Service", "Scanning Errors", "Customer Service-Misc", "Empties and Additionals")
),
valid_transactions AS (
    SELECT *
    FROM `machine_learning.transactions` a 
    join product_filter b on a.prod_id  = b.prod_id 
    WHERE trans_dt < "2020-03-01"
    AND a.prod_id IN (SELECT prod_id FROM product_filter)
        AND 
        -- Logic 1: Either sales_qty or sales_wgt is zero, but not both
        ((sales_qty = 0 AND sales_wgt <> 0) OR (sales_qty <> 0 AND sales_wgt = 0))
        AND 
        -- Logics 2 and 3 are parallel conditions
        (
            (prod_category NOT IN ("Coupon", "returns") AND (sales_qty > 0 OR sales_wgt > 0))
            OR
            (prod_category IN ("Coupon", "returns") AND (sales_qty < 0 OR sales_wgt < 0))
        )
    AND sales_amt >= 0
),
transactions_per_day AS (
    SELECT cust_id, trans_dt, COUNT(DISTINCT trans_id) AS trans_per_day
    FROM valid_transactions
    GROUP BY cust_id, trans_dt
    HAVING trans_per_day <= 10
),
eligible_custs AS (
    SELECT v.cust_id
    FROM valid_transactions v
    JOIN transactions_per_day tpd ON v.cust_id = tpd.cust_id AND v.trans_dt = tpd.trans_dt
    GROUP BY v.cust_id
    HAVING COUNT(DISTINCT v.trans_id) >= 5
    AND COUNT(DISTINCT v.trans_dt) >= 5
    AND COUNT(v.trans_id) <= 20000
),
sampled_custs AS (
    SELECT cust_id
    FROM eligible_custs
    WHERE MOD(ABS(FARM_FINGERPRINT(CAST(cust_id AS STRING))), 200) < 1
)
SELECT tx.*
FROM `valid_transactions` tx
JOIN sampled_custs ON tx.cust_id = sampled_custs.cust_id
WHERE tx.trans_dt < "2020-03-01"
"""

# Execute the query
query_job = client.query(QUERY)  # API request

# Convert to DataFrame
sample_transaction = query_job.to_dataframe()  # Waits for query to finish and converts it to DataFrame


In [ ]:
# Load product and profit margin data
product_profit_margin_df = pd.read_excel("C:/Users/ctlan/OneDrive/desktop/AI at Scale/HW/Product Category Profit Margin.xlsx")

# Merge with product_profit_margin to get profit margins
merged_df = pd.merge(sample_transaction, product_profit_margin_df, on='prod_category', how='left')

# Calculate profit for each transaction
merged_df['profit'] = merged_df['sales_amt'] * merged_df['profit_margin']

## loyal customer analysis

In [ ]:
import pandas as pd

# Assuming merged_df is prepared with columns 'cust_id' and 'profit'

# Mark loyal customers
merged_df['is_loyal'] = merged_df['cust_id'].apply(lambda x: len(str(x)) == 10)

# Calculate numbers of loyal and total customers
total_customers = merged_df['cust_id'].nunique()
loyal_customers = merged_df[merged_df['is_loyal']]['cust_id'].nunique()

# Calculate total and loyal customers' profit
total_profit = merged_df['profit'].sum()
loyal_customers_profit = merged_df[merged_df['is_loyal']]['profit'].sum()

# Calculate proportions
loyal_customers_ratio = loyal_customers / total_customers
loyal_customers_profit_ratio = loyal_customers_profit / total_profit

# Create the output DataFrame
output_df = pd.DataFrame({
    'Metric': ['Loyal Customer Ratio', 'Loyal Customer Profit Ratio'],
    'Value': [loyal_customers_ratio, loyal_customers_profit_ratio]
})

output_df

## customer pattern analysis

In [ ]:
import pandas as pd

# Adjustments for accurately identifying customers by category

# Assuming merged_df is prepared with 'cust_id', 'prod_category', and 'sales_amt'

# Identify all unique customers
all_customers = merged_df['cust_id'].unique()

# 1. Never use coupon
# Find customers who have used coupons
customers_used_coupons = merged_df[merged_df['prod_category'] == 'Coupons']['cust_id'].unique()
# Find customers who have never used a coupon by excluding those who have
customers_never_use_coupon = set(all_customers) - set(customers_used_coupons)
# Calculate their profit
profit_never_use_coupon = merged_df[(merged_df['cust_id'].isin(customers_never_use_coupon)) & (~merged_df['prod_category'].str.contains('Coupons'))]['sales_amt'].sum()

# 2. Only buy One/Two/Three certain categories
# Count unique categories per customer
cust_category_counts = merged_df.groupby('cust_id')['prod_category'].nunique()
# Customers who only buy from 1, 2, or 3 categories
customers_one_two_three_categories = cust_category_counts[(cust_category_counts >= 1) & (cust_category_counts <= 3)].index
# Calculate their profit
profit_one_two_three_categories = merged_df[merged_df['cust_id'].isin(customers_one_two_three_categories)]['sales_amt'].sum()

# 3. Buy everything (50+ categories)
customers_buy_everything = cust_category_counts[cust_category_counts >= 50].index
# Calculate their profit
profit_buy_everything = merged_df[merged_df['cust_id'].isin(customers_buy_everything)]['sales_amt'].sum()

# Total numbers and profits
total_customers = len(all_customers)
total_profit = merged_df['sales_amt'].sum()

# Calculate proportions
data = {
    'Category': ['Never Use Coupon', 'Buy 1-3 Categories', 'Buy 50+ Categories'],
    'Customer Count Proportion': [
        len(customers_never_use_coupon) / total_customers,
        len(customers_one_two_three_categories) / total_customers,
        len(customers_buy_everything) / total_customers
    ],
    'Profit Proportion': [
        profit_never_use_coupon / total_profit,
        profit_one_two_three_categories / total_profit,
        profit_buy_everything / total_profit
    ]
}

# Create the output DataFrame
output_df = pd.DataFrame(data)

output_df


## cherry picker

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 假设sample_transaction是已经加载的DataFrame，其中包含你提供的所有列。

# 首先，我们将计算每个cust_id对于prod_category为Coupons的transaction占比
# 1. 标记每个transaction是否为Coupons
sample_transaction['is_coupon'] = (sample_transaction['prod_category'] == 'Coupons').astype(int)

# 2. 对每个cust_id，计算总transaction数量和Coupons transaction的数量
cust_transactions_summary = sample_transaction.groupby('cust_id').agg(
    total_transactions=('trans_id', 'nunique'),  # 计算每个客户的唯一交易数
    coupons_transactions=('is_coupon', 'sum')  # 计算Coupons的交易数
)

# 3. 计算每个cust_id的Coupons transaction占比
cust_transactions_summary['coupons_ratio'] = cust_transactions_summary['coupons_transactions'] / cust_transactions_summary['total_transactions']

# 4. 绘制分布直方图，纵坐标为顾客数的比例
# 计算直方图
counts, bins = np.histogram(cust_transactions_summary['coupons_ratio'], bins=np.arange(0, 1.05, 0.05))

# 将计数转换为比例
counts = counts / counts.sum()

# 绘制直方图
plt.bar(bins[:-1], counts, width=0.05, align='edge', edgecolor='black')
plt.xlabel('Coupons Transaction Ratio')
plt.ylabel('Proportion of Customers')
plt.title('Distribution of Coupons Transaction Ratio per Customer')
plt.xticks(np.arange(0, 1.1, 0.1))
plt.show()



## bar chart 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming merged_df is prepared with 'cust_id', 'prod_category', and 'sales_amt'

# Calculate the number of unique categories purchased by each customer
category_counts_per_customer = merged_df.groupby('cust_id')['prod_category'].nunique()

# Calculate the total profit by customer
profit_per_customer = merged_df.groupby('cust_id')['sales_amt'].sum()

# Merge the two series into a DataFrame for easier analysis
customer_analysis_df = pd.DataFrame({
    'Category_Count': category_counts_per_customer,
    'Total_Profit': profit_per_customer
})

# Define bins for the category count histogram
bins_category_count = pd.cut(customer_analysis_df['Category_Count'], bins=5, labels=['1-5', '6-10', '11-15', '16-20', '21+'])

# Plot histogram for the distribution of customers by category count
plt.figure(figsize=(10, 6))
bins_category_count.value_counts(normalize=True).sort_index().plot(kind='bar', color='red')
plt.title('Customer Distribution by Number of Product Categories Purchased')
plt.xlabel('Number of Product Categories')
plt.ylabel('Proportion of Customers')
plt.show()

# For profit contribution, first assign each customer to a bin based on their category count
customer_analysis_df['Category_Count_Bin'] = bins_category_count

# Then, calculate the total profit for each bin
profit_contribution_by_bin = customer_analysis_df.groupby('Category_Count_Bin')['Total_Profit'].sum()

# Normalize the profit contribution by the total profit to get the proportion
profit_contribution_by_bin_normalized = profit_contribution_by_bin / profit_contribution_by_bin.sum()

# Plot histogram for the profit contribution by category count bin
plt.figure(figsize=(10, 6))
profit_contribution_by_bin_normalized.plot(kind='bar', color='red')
plt.title('Profit Contribution by Number of Product Categories Purchased')
plt.xlabel('Number of Product Categories')
plt.ylabel('Proportion of Profit')
plt.show()


## T -test check the distribution 

In [ ]:
from statsmodels.stats.proportion import proportions_ztest
import pandas as pd

# Create a DataFrame with the provided data
data = {
    'Category': ['Never Use Coupon', 'Buy 1-3 Categories', 'Buy 50+ Categories'],
    'Customer Count Proportion 0.1%': [0.484003, 0.013765, 0.083333],
    'Profit Proportion 0.1%': [0.146583, 0.003421, 0.451492],
    'Customer Count Proportion 0.5%': [0.489452, 0.012824, 0.075960],
    'Profit Proportion 0.5%': [0.142310, 0.002219, 0.443786]
}

df = pd.DataFrame(data)

# We need to assume a total number of customers to convert proportions to counts
# Let's assume there are 1000 customers in each group for demonstration purposes
n_customers = 1000

# Calculate the observed counts of customers from the proportions
df['Customer Count 0.1%'] = df['Customer Count Proportion 0.1%'] * n_customers
df['Customer Count 0.5%'] = df['Customer Count Proportion 0.5%'] * n_customers

# Extract the counts for the category 'Buy 50+ Categories' to perform the z-test
count = df.loc[df['Category'] == 'Buy 50+ Categories', ['Customer Count 0.1%', 'Customer Count 0.5%']].values.flatten()
nobs = [n_customers, n_customers]  # The number of observations in each sample

# Perform the two-proportion z-test
stat, pval = proportions_ztest(count, nobs)

stat, pval


In [ ]:
# Initialize a list to hold the z-test results
ztest_results = []

# There are three comparisons to make:
# 1. Never Use Coupon
# 2. Buy 1-3 Categories
# 3. Buy 50+ Categories

# Loop through each category to perform the z-tests
for category in df['Category']:
    # Extract the counts for the current category to perform the z-test
    count = df.loc[df['Category'] == category, ['Customer Count 0.1%', 'Customer Count 0.5%']].values.flatten()
    
    # Perform the two-proportion z-test
    stat, pval = proportions_ztest(count, nobs)
    
    # Append the results to our list
    ztest_results.append((category, stat, pval))

# Convert the z-test results to a DataFrame
output_df = pd.DataFrame(ztest_results, columns=['Category', 'Z-Score', 'P-Value'])

output_df
